In [1]:
addprocs(CPU_CORES)

@everywhere begin
  using StochasticDiffEq, DiffEqProblemLibrary, ParallelDataTransfer
  srand(99 + myid())
  prob = oval2ModelExample(largeFluctuations=true,useBigs=false)
  prob.tspan = (0.0,1.0)
  println("Solve once to compile.")
  sol = solve(prob,EM(),dt=1/2^(18),adaptive=false,save_timeseries=false)
  sol = solve(prob,RKMil(),dt=1/2^(18),adaptive=false,save_timeseries=false)
  sol = solve(prob,SRIW1(),dt=1/2^(18),adaptive=false,save_timeseries=false)
  Int(sol.u[1]!=NaN)
  println("Compilation complete.")
  js = 16:21
  dts = 1./2.^(js)
  fails = Array{Int}(length(dts),3)
  times = Array{Float64}(length(dts),3)
  numRuns = 10000
end
println("Setup Complete")

  likely near In[1]:1
AbstractSDEProblem{#uType<:Any, #tType<:AnyWARNING: Method definition , #solve#isinplace<:(AnyArray, {#NoiseClass<:AnyAny, , 1#F<:}Any, , DiffEqBase#F2<:.Any#solve, , #F3<:DiffEqBaseAny.}AbstractSDEProblem, {#algType<:#uType<:DiffEqBaseAny., AbstractSDEAlgorithm#tType<:Any),  in module StochasticDiffEq#isinplace<:Any,  at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:25#NoiseClass<: overwrittenAny in module StochasticDiffEq,  at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:25#F<:.
Any, #F2<:Any, #F3<:Any}WARNING: Method definition , solve#algType<:(DiffEqBaseDiffEqBase..AbstractSDEAlgorithmAbstractSDEProblem, {Any, #uType<:AnyAny),  in module StochasticDiffEq#tType<: overwrittenAny in module StochasticDiffEq, .
#isinplace<:Any, #NoiseClass<:Any, #F<:Any, WARNING: Method definition #F2<:#solveAny(, Array#F3<:{AnyAny, 1}}, , #algType<:DiffEqBaseDiffEqBase..#solveAbstractSDEAlgorithm, , DiffEqBaseAny.)AbstractSDEProblem{ in module StochasticDif

	From worker 13:	Solve once to compile.
	From worker 12:	Solve once to compile.
	From worker 5:	Solve once to compile.
	From worker 14:	Solve once to compile.
	From worker 8:	Solve once to compile.
	From worker 16:	Solve once to compile.
	From worker 9:	Solve once to compile.
	From worker 17:	Solve once to compile.
	From worker 10:	Solve once to compile.
	From worker 7:	Solve once to compile.
	From worker 2:	Solve once to compile.
	From worker 15:	Solve once to compile.
	From worker 6:	Solve once to compile.
	From worker 3:	Solve once to compile.
	From worker 11:	Solve once to compile.
	From worker 4:	Solve once to compile.
	From worker 12:	Compilation complete.
	From worker 14:	Compilation complete.
	From worker 13:	Compilation complete.
	From worker 5:	Compilation complete.
	From worker 8:	Compilation complete.
	From worker 16:	Compilation complete.
	From worker 9:	Compilation complete.
	From worker 10:	Compilation complete.
	From worker 7:	Compilation complete.
	From worker 15:	Comp

Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Expr, Expr, Expr, Expr, Expr, Expr, Expr, Expr, Expr, Expr, Expr, Array{Symbol, 1}, Float64, Float64, Float64) in module ParameterizedFunctions at /home/crackauc/.julia/v0.5/ParameterizedFunctions/src/maketype.jl:25 overwrittenWARNING: Method definition  at /home/crackauc/.julia/v0.5/ParameterizedFunctions/src/maketype.jl:25(::.
Type{ParameterizedFunctions.LotkaVolterra}WARNING: Method definition )((::Any, TypeAny{, AnyParameterizedFunctions, .AnyLotkaVolterra, }Any), (AnyExpr, , Any, ArrayAny{, ExprAny, , Any1, }, Any, ArrayAny{, AnySymEngine, .AnyBasic, , Any1, }Any, , ArrayAny{, ExprAny, , 1Any}, , AnyArray, {AnyExpr, , WARNING: Method definition Any1, (::Any}Type, , {AnyArrayParameterizedFunctions.{, LotkaVolterraSymbolAny}, , )Any1(, })Any, , ArrayAny, Any, Any) in module ParameterizedFunctionsWARNING: Method d

Solve once to compile.
Compilation complete.
Setup Complete


In [2]:
## Timing Runs

@everywhere function runAdaptive(i)
  sol = solve(prob,SRIW1(),dt=1/2^(18),adaptiveAlg=:RSwM3,abstol=2.0^(-14),reltol=2.0^(-9),maxIters=Int(1e10),qmax=1.125,save_timeseries=false)
  Int(any(isnan,sol[end]))
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptive,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 28.694792643


In [3]:
for j in eachindex(js)
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere function runEM(i,j)
    sol =solve(prob,EM(),dt=dts[j],maxIters=Int(1e11),save_timeseries=false)
    Int(any(isnan,sol.u[end]))
  end
  @everywhere srand(99 + myid())
  t1 = @elapsed numFails = sum(pmap((i)->runEM(i,j),1:numRuns))
  println("The number of Euler-Maruyama Fails is $numFails. Elapsed time was $t1")
  fails[j,1] = numFails
  times[j,1] = t1
end

j = 1


The number of Euler-Maruyama Fails is 142. Elapsed time was 44.73824596
j = 2


runEM(Any, Any) in module Main at In[3]:5 overwritten at In[3]:5.
 at In[3]:5 overwritten at In[3]:5.
 at In[3]:5.


The number of Euler-Maruyama Fails is 39. Elapsed time was 87.3244607
j = 3


 at In[3]:5 overwritten at In[3]:5.
(AnyAny), WARNING: Method definition  in module MainAny) at In[3]:5runEM in module Main( overwrittenAny at In[3]:5,  at In[3]:5 overwrittenAny.
) at In[3]:5 in module Main.
 at In[3]:5 overwritten at In[3]:5.


...

The number of Euler-Maruyama Fails is 1. Elapsed time was 336.674466557
j = 5


.
runEM(AnyWARNING: Method definition , Any)runEM in module Main( at In[3]:5Any overwritten,  at In[3]:5Any.
) in module Main at In[3]:5 overwritten at In[3]:5.
 overwritten at In[3]:5.
 in module Main at In[3]:5 overwritten at In[3]:5.


The number of Euler-Maruyama Fails is 0. Elapsed time was 669.968370265
j = 6


runEM(Any, Any) in module Main at In[3]:5 overwritten at In[3]:5.
Any) in module Main at In[3]:5 overwritten at In[3]:5.
 at In[3]:5.
, Any) in module MainWARNING: Method definition  at In[3]:5 overwrittenrunEM at In[3]:5(.
Any, Any) in module Main at In[3]:5 overwritten at In[3]:5.


The number of Euler-Maruyama Fails is 0. Elapsed time was 1342.804040781


In [2]:
for j in 2:4
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere function runSRI(i,j)
    sol =solve(prob,SRIW1(),dt=dts[j],adaptive=false,save_timeseries=false)
    Int(any(isnan,sol.u[end]))
  end
  @everywhere srand(99 + myid())
  t2 = @elapsed numFails = sum(pmap((i)->runSRI(i,j),1:numRuns))
  println("The number of Rossler-SRI Fails is $numFails. Elapsed time was $t2")
  fails[j,2] = numFails
  times[j,2] = t2
end

j = 2


The number of Rossler-SRI Fails is 23. Elapsed time was 126.408791525
j = 3


 overwritten at In[2]:5.
runSRI at In[2]:5(.
Any, Any) in module Main at In[2]:5 overwritten at In[2]:5.
.
 at In[2]:5 overwritten, WARNING: Method definition  at In[2]:5Any.
) in module MainrunSRI at In[2]:5 overwritten( at In[2]:5.
Any, Any) in module Main at In[2]:5 overwritten at In[2]:5.


The number of Rossler-SRI Fails is 0. Elapsed time was 245.507665296
j = 4


 overwritten at In[2]:5.
Any, Any) in module Main at In[2]:5 overwritten at In[2]:5.
 at In[2]:5 overwritten at In[2]:5.


The number of Rossler-SRI Fails is 0. Elapsed time was 489.964318008


In [ ]:
for j in eachindex(js)
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere function runMil(i,j)
    sol =solve(prob,RKMil(),dt=dts[j])
    Int(any(isnan,sol.u[end]))
  end
  @everywhere srand(99 + myid())
  t3 = @elapsed numFails = sum(pmap((i)->runMil(i,j),1:numRuns))
  println("The number of RK-Milstein Fails is $numFails. Elapsed time was $t3")
  fails[j,3] = numFails
  times[j,3] = t3
end

j = 1


The number of RK-Milstein Fails is 143. Elapsed time was 137.056656672
j = 2


AnyrunMil)(Any in module Main, Any) at In[3]:5 in module Main overwritten at In[3]:5 at In[3]:5 overwritten at In[3]:5.
.
runMil(Any, Any) in module Main at In[3]:5 overwritten at In[3]:5.


The number of RK-Milstein Fails is 37. Elapsed time was 290.997939739
j = 3


runMil(Any, Any) in module Main at In[3]:5 overwritten at In[3]:5.
 overwritten, Any at In[3]:5).
 in module Main at In[3]:5 overwritten at In[3]:5.
runMil(Any, Any) in module Main at In[3]:5 overwritten at In[3]:5.


...

runMil(Any, Any) in module Main at In[3]:5 overwrittenWARNING: Method definition  at In[3]:5.
runMil(Any, Any) in module Main at In[3]:5 overwritten at In[3]:5.
) in module Main at In[3]:5 overwritten at In[3]:5.
Any) in module Main at In[3]:5 overwritten at In[3]:5.


The number of RK-Milstein Fails is 1. Elapsed time was 1652.250690362
j = 5


 at In[3]:5.
runMil(Any, Any) in module Main at In[3]:5 overwritten at In[3]:5.


The number of RK-Milstein Fails is 0. Elapsed time was 4233.264495831
j = 6


 at In[3]:5.
) in module Main at In[3]:5 overwritten at In[3]:5.


In [ ]:
using Plots
lw = 3
p2 = plot(dts,times,xscale=:log2,yscale=:log2,guidefont=font(16),tickfont=font(14),yguide="Elapsed Time (s)",xguide=L"Chosen $\Delta t$",top_margin=50px,linewidth=lw,lab=["Euler-Maruyama" "RK-Mil" "RosslerSRI"],legendfont=font(14))
plot!(dts,repmat([adaptiveTime],11),linewidth=lw,line=:dash,lab="ESRK+RSwM3",left_margin=75px)
scatter!([2.0^(-20);2.0^(-20);2.0^(-18)],[times[5,1];times[5,2];times[3,3]],markersize=20,c=:red,lab="")
plot(p2,size=(800,800))